## Molecular dynamics simulation of the paper of Kim 
Skjegstad, L. E. J., Nickels, J. F., Sneppen, K. & Kirkegaard, J. B. Epigenetic switching with asymmetric bridging interactions. Biophysical Journal 122, 2421–2429 (2023).


In [51]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
#from scipy.optimize import fsolve
from scipy.special import lambertw

"The dynamics of the chromatin is described by an over-damped Langevin equation of the form: 
$$ d\textbf{X}_i(t) = - \nabla_{X_i} U (\textbf{X}(t)) dt + d\textbf{W}_i(t) $$
where $\textbf{X}_i$ is the position of the i’th monomer, U is the aggregate potential function, and $\textbf{W}$ is a Wiener process satisfying $<\textbf{W}_i(t) , \textbf{W}_i(t')> = 3\sigma^2\delta(t-t')$, with $\sigma$ setting the diffusive noise scale."

"The potential U depends on the positions and states of all other monomers : 
$$ U \propto [\sum_{ij} e^{-4r_{ij}/{l_0}} - \sum_{ij  \in  bonds} e^{-4br_{ij}/{l_0}}]$$

Here, the terms in the first summation account for repulsion between all monomers, and the terms in the second summation account for attraction for the S-state monomers that form bonds. For bonded monomers, the two exponential terms create a potential well with an equilibrium distance equal to $l_0/2$, ensured by setting $b = - LambertW(-2e^{-2})/2$. We note that U, as specified here, does not constitute the total potential, as, again, adjacent monomers in the polymer are constrained to be distance $l_0$ from one another."

As in the article we will use the Euler-Marayuma algorithm to resolve the 1st equation

In [52]:
def compute_b_lambertw(l0):
    b_solution = lambertw(-2 * np.exp(-2)) / 2
    return b_solution.real

In [69]:
def potential_energy(positions, bonds, l0, b):
    repulsion_term = np.sum(np.exp(-4 * np.abs(positions[:, np.newaxis] - positions) / l0), dtype=np.float128)
    bonding_term = np.sum(np.exp(-4 * b * np.abs(positions[:, np.newaxis] - positions[bonds]) / l0), dtype=np.float128)
    #
    return repulsion_term - bonding_term

In [54]:
def potential_gradient(positions, bonds, l0, b):
    repulsion_gradient = -4 * np.sum(np.sign(positions[:, np.newaxis] - positions) *
                                     np.exp(-4 * np.abs(positions[:, np.newaxis] - positions) / l0) / l0, axis=1, dtype=np.float128)

    bonding_gradient = 4 * b * np.sum(np.sign(positions[:, np.newaxis] - positions[bonds[:, 0]]) *
                                          np.exp(-4 * b * np.abs(positions[:, np.newaxis] - positions[bonds[:, 0]]) / l0) / l0, axis=1, dtype=np.float128)

    #bonding_gradient = 0
    return repulsion_gradient - bonding_gradient


In [70]:
# Function for Langevin dynamics simulation
def langevin_dynamics(positions, bonds, l0, b, diffusion_coefficient, time_step, num_steps):
    num_particles = len(positions)
    trajectory = [positions.copy()]

    for _ in range(num_steps):
        # Compute the noise term
        noise = np.random.normal(0, np.sqrt(6 * diffusion_coefficient * time_step), (num_particles,3))

        # Compute the potential gradient
        grad_potential = potential_gradient(positions, bonds, l0, b)

        # Update positions using overdamped Langevin dynamics
        positions = positions - grad_potential * time_step + noise

        # Store the updated positions in the trajectory
        trajectory.append(positions.copy())

    return np.array(trajectory)


In [71]:
def linear_initial_positions(num_particles,dimension):
    startpts = np.array([1, 1, 1])
    endpts = np.array(dimension*[num_particles])

    return np.linspace(start=startpts, stop=endpts, num=num_particles)


In [72]:
# Parameters
dimension = 3
num_particles = 10
diffusion_coefficient = 1.0
time_step = 0.01
num_steps = 1000
l0 = 1.0  # equilibrium distance
b = compute_b_lambertw(l0)

# Example bonded pairs
bonds = np.array([[i, (i + 1) % num_particles] for i in range(num_particles - 1)])

# Initial positions (you can set these according to your problem)
initial_positions = linear_initial_positions(num_particles,dimension)

# Run Langevin dynamics simulation
trajectory = langevin_dynamics(initial_positions, bonds, l0, b, diffusion_coefficient, time_step, num_steps)

# Save the trajectory to an XYZ file
with open('langevin_trajectory.xyz', 'w') as f:
    for frame in trajectory:
        f.write(f"{num_particles}\n")
        f.write("Generated by Langevin Dynamics\n")
        for i in range(num_particles):
            f.write(f"P {frame[i, 0]} {frame[i, 1]} {frame[i, 2]}\n")


/var/folders/jg/gqqb9c4s6qd3ftqhm6____m80000gn/T/ipykernel_52864/3713077086.py:6: RuntimeWarning: overflow encountered in exp
  np.exp(-4 * b * np.abs(positions[:, np.newaxis] - positions[bonds[:, 0]]) / l0) / l0, axis=1, dtype=np.float128)
/var/folders/jg/gqqb9c4s6qd3ftqhm6____m80000gn/T/ipykernel_52864/3713077086.py:2: RuntimeWarning: invalid value encountered in subtract
  repulsion_gradient = -4 * np.sum(np.sign(positions[:, np.newaxis] - positions) *
/var/folders/jg/gqqb9c4s6qd3ftqhm6____m80000gn/T/ipykernel_52864/3713077086.py:3: RuntimeWarning: invalid value encountered in subtract
  np.exp(-4 * np.abs(positions[:, np.newaxis] - positions) / l0) / l0, axis=1, dtype=np.float128)
/Users/adrienberard/opt/anaconda3/envs/myenv2/lib/python3.12/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/var/folders/jg/gqqb9c4s6qd3ftqhm6____m80000gn/T/ipykernel_52864/3713077086.py:5: RuntimeW

Do a polymer chain with relations between the neighbors 

In [68]:
initial_positions

array([[ 1.,  1.,  1.],
       [ 2.,  2.,  2.],
       [ 3.,  3.,  3.],
       [ 4.,  4.,  4.],
       [ 5.,  5.,  5.],
       [ 6.,  6.,  6.],
       [ 7.,  7.,  7.],
       [ 8.,  8.,  8.],
       [ 9.,  9.,  9.],
       [10., 10., 10.]])